In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import natsort
import glob

In [ ]:
CHECKPOINT_DIR = ''
DATA_DIR = '/content/drive/MyDrive/아이펠/PoCLab/classification/dataset'
TEST_IMG_DIR = ''

In [ ]:
data_dir = DATA_DIR
# batch_size = 16
batch_size = 64
img_height = 256
img_width = 256
num_classes = 3

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  shuffle=True,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
class_names = train_ds.class_names
print(class_names)

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
normalization_layer = layers.Rescaling(1./255)

In [ ]:
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x, training=True), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]

print(np.min(first_image), np.max(first_image))

0.0 0.9991542


In [ ]:
len(class_names)

10

In [ ]:
data_augmentation = keras.Sequential(
  [
    layers.RandomFlip("horizontal",
                      input_shape=(img_height,
                                  img_width,
                                  3)),
    layers.RandomRotation(0.2),
    #layers.RandomZoom(0.1),
  ]
)

In [ ]:
aug_model = Sequential([
    data_augmentation
])

In [ ]:
aug_model.add(tf.keras.applications.resnet.ResNet101(
    include_top=True,
    weights=None,
    input_tensor=None,
    input_shape=(img_height, img_width,3),
    pooling=max,
    classes=len(class_names),
))

In [ ]:
from keras import optimizers
sgd = tf.keras.optimizers.SGD(learning_rate=0.001) # lr 수정부분

aug_model.compile(optimizer = sgd,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
aug_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 256, 256, 3)       0         
                                                                 
 resnet101 (Functional)      (None, 10)                42678666  
                                                                 
Total params: 42,678,666
Trainable params: 42,573,322
Non-trainable params: 105,344
_________________________________________________________________


In [ ]:
checkpoint_path = CHECKPOINT_DIR +'/try-{epoch}.ckpt"

cp_callback = tf.keras.callbacks.ModelCheckpoint(
  filepath=checkpoint_path,
  monitor='val_accuracy',
  verbose=1,
  save_best_only=True,
  mode='auto')

In [ ]:

epochs=100

history = aug_model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs,
  callbacks=[cp_callback]
)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
def predict_img(img_path):
    img_path = img_path
    img_path = img_path.replace('\\','/')
    img = tf.keras.utils.load_img(
        img_path, target_size=(256, 256)
    )
    img_array = tf.keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0) # Create a batch

    predictions = aug_model.predict(img_array)
    score = tf.nn.softmax(predictions[0])

    prediction, acc = class_names[np.argmax(score)], 100 * np.max(score)
    
    print(f'{round(acc)}% 확률로 {prediction}입니다.')
    return prediction, acc


In [ ]:
for i in glob.glob(TEST_IMG_DIR + '/*'): 
    predict_img(i)